In [1]:
# Do some imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matlab.engine



# Load data and process data
data_dir = "../data/"

In [2]:
#epsilon = 1
#conn = '8m'
numberimfs = 2
eng = matlab.engine.start_matlab()

In [3]:
def load_data(data_dir):
    train = pd.read_json(data_dir+"test.json")
    # Fill 'na' angles with zero
    train.inc_angle = train.inc_angle.replace('na', 0)
    train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
    return train

train = load_data(data_dir)

In [4]:
# Process data into images
def process_images(df):
    X_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in df["band_1"]])
    X_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in df["band_2"]])
    # Merge bands and add another band as the mean of Band 1 and Band 2 (useful for the ImageDataGenerator later)
    imgs = np.concatenate([X_band1[:, :, :, np.newaxis]
                            , X_band2[:, :, :, np.newaxis]
                            ,((X_band1+X_band2)/2)[:, :, :, np.newaxis]], axis=-1)
    return imgs

X_train = process_images(train)

In [5]:
# Calculate IMFs, residue, and medel(?) using matlab wrapper
def IEMD(img, epsilon=0.1, numberimfs=4, conn='8m'):
    matlab_arr = matlab.double(img.tolist())
    matlab_imfs, matlab_residue, matlab_medel = eng.IEMD_public(matlab_arr, epsilon, numberimfs, conn, nargout=3) 
    
    imfs = np.asarray(matlab_imfs)
    residue = np.asarray(matlab_residue)
    medel = np.asarray(matlab_medel)
    return imfs, residue, medel

In [6]:
# Create the columns in the pandas object for each IMF
for band in range(1,3):
    for imf in range(1,numberimfs + 1):
        column_name = "band_" + str(band) + "_imf_" + str(imf)
        #train[column_name] = np.nan
        train[column_name] = np.empty((len(train), 0)).tolist()
        #train[column_name] = np.zeros((75,75)).reshape(1, 75 * 75).tolist()

In [12]:
# Calculate every IMF for every image
from timeit import default_timer as timer
def IMF_to_JSON(data):
    num_img = len(data)
    start_time = timer()
    for i, images in enumerate(data):
        band1_imf, _, _ = IEMD(images[:,:,0], 1, 2)
        band2_imf, _, _ = IEMD(images[:,:,1], 1, 2)

        for imf in range(numberimfs):
            column_name_band1 = "band_1_imf_" + str(imf + 1)
            column_name_band2 = "band_2_imf_" + str(imf + 1)
            train.set_value(i, column_name_band1, band1_imf[:,:,imf].reshape(75 * 75).tolist())
            train.set_value(i, column_name_band2, band2_imf[:,:,imf].reshape(75 * 75).tolist())

        msg = "Image " + str(i) + "/" + str(num_img) + " completed..."
        print (msg, end="\r")
    print("Total run time: " + str(timer() - start_time))
    print("Saving to JSON...")
    train.to_json("../data/train_with_imfs.json")
    print("Done!")

In [ ]:
IMF_to_JSON(X_train)